In [1]:
import time
from copy import deepcopy
from typing import Callable, Any

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [2]:
import rectools
from rectools.dataset import Interactions, Dataset
from rectools.model_selection import Splitter
from rectools.models.base import ModelBase
from rectools.models import RandomModel, PopularModel
from rectools.metrics.base import MetricAtK
from rectools.metrics import (
    Precision,
    Recall,
    MAP,
    NDCG,
    MeanInvUserFreq,
    calc_metrics,
)

# Read Data

In [3]:
interactions_df = pd.read_csv("../datasets/interactions.csv", parse_dates=["last_watch_dt"])
interactions_df.rename(
    columns={"last_watch_dt": rectools.Columns.Datetime, "total_dur": rectools.Columns.Weight},
    inplace=True
)
interactions = Interactions(interactions_df)
interactions.df.head()

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250.0,72.0
1,699317,1659,2021-05-29,8317.0,100.0
2,656683,7107,2021-05-09,10.0,0.0
3,864613,7638,2021-07-05,14483.0,100.0
4,964868,9506,2021-04-30,6725.0,100.0


In [4]:
items = pd.read_csv("../datasets/items.csv")
items.head()

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


# Validation

In [5]:
def add_thresholds(metrics: dict[str, Callable | tuple[Callable, dict[str, Any]]], ks: list[int]):
    metrics_with_thresholds = {}
    for name, metric in metrics.items():
        kwargs = {}
        if isinstance(metric, tuple):
            kwargs.update(**metric[1])
            metric = metric[0]
        for k in ks:
            metrics_with_thresholds.update({f"{name}@{k}": metric(k=k, **kwargs)})
    return metrics_with_thresholds

In [6]:
def calc_coverage(reco):
    return reco.item_id.nunique() / len(reco)

In [7]:
def cross_val(
    dataset: pd.DataFrame,
    models: dict[str, ModelBase],
    metrics: dict[str, MetricAtK | Callable],
    splitter: Splitter,
    k: int,
    num_splits: int = 0,
):
    rectool_metrics = {k: v for k, v in metrics.items() if isinstance(v, MetricAtK)}
    custom_metrics = {k: v for k, v in metrics.items() if k not in rectool_metrics}
    metric_values = []
    interactions = Interactions(dataset)
    pbar = tqdm(total=len(models) * num_splits)
    pbar.set_description(f"splitting")
    splits = splitter.split(interactions)
    for train_ids, test_ids, i in splits:
        num_fold = i["i_split"]
        train = Dataset.construct(dataset.iloc[train_ids])
        test = Dataset.construct(dataset.iloc[test_ids])

        for model_name, orig_model in models.items():
            pbar_prefix = f"fold {num_fold}, {model_name}"
            pbar.set_description(f"{pbar_prefix} training")
            model = deepcopy(orig_model)
            start = time.time()
            model.fit(train)
            end = time.time()
            pbar.set_description(f"{pbar_prefix} predicting")
            reco = model.recommend(test.user_id_map.external_ids, train, k, True)
            del model
            pbar.set_description(f"{pbar_prefix} evaluating")
            cur_metrics = {
                "model": model_name,
                "time": end - start,
                **calc_metrics(
                    rectool_metrics,
                    reco=reco,
                    interactions=test.interactions.df,
                    prev_interactions=train.interactions.df,
                ),
            }
            for name, metric in custom_metrics.items():
                cur_metrics.update({name: metric(reco)})
            metric_values.append(cur_metrics)
            pbar.update(1)
    pbar.close()
    return pd.DataFrame(metric_values).groupby("model").mean()

In [8]:
metrics = add_thresholds(
    {
        "precision": Precision,
        "recall": Recall,
        "MAP": MAP,
        "NDCG": NDCG,
        "novelty": MeanInvUserFreq,
    },
    [1, 5, 10],
)
metrics.update({"coverage": calc_coverage})
metrics

{'precision@1': Precision(k=1),
 'precision@5': Precision(k=5),
 'precision@10': Precision(k=10),
 'recall@1': Recall(k=1),
 'recall@5': Recall(k=5),
 'recall@10': Recall(k=10),
 'MAP@1': MAP(k=1, divide_by_k=False),
 'MAP@5': MAP(k=5, divide_by_k=False),
 'MAP@10': MAP(k=10, divide_by_k=False),
 'NDCG@1': NDCG(k=1, log_base=2),
 'NDCG@5': NDCG(k=5, log_base=2),
 'NDCG@10': NDCG(k=10, log_base=2),
 'novelty@1': MeanInvUserFreq(k=1),
 'novelty@5': MeanInvUserFreq(k=5),
 'novelty@10': MeanInvUserFreq(k=10),
 'coverage': <function __main__.calc_coverage(reco)>}

In [9]:
NUM_RECOS = 10
NUM_SPLITS = 3
SEED = 32
splitter = rectools.model_selection.time_split.TimeRangeSplitter("1D", NUM_SPLITS)
models = {"random": RandomModel(random_state=SEED), "popular": PopularModel()}

results = cross_val(interactions_df, models, metrics, splitter, NUM_RECOS, num_splits=NUM_SPLITS)

  0%|          | 0/6 [00:00<?, ?it/s]

In [10]:
results

,time,precision@1,recall@1,precision@5,recall@5,precision@10,recall@10,NDCG@1,NDCG@5,NDCG@10,MAP@1,MAP@5,MAP@10,novelty@1,novelty@5,novelty@10,coverage
model,,,,,,,,,,,,,,,,,
popular,5.251233,0.0,0.0,0.0,0.0,0.000001,0.000003,0.0,0.0,7.181475e-07,0.0,0.0,4.309706e-07,16.810539,16.168155,15.207510,0.000110
random,0.000000,0.0,0.0,0.0,0.0,0.000002,0.000011,0.0,0.0,1.505503e-06,0.0,0.0,1.401207e-06,15.861961,15.860099,15.860982,0.049638


# Visualization

In [11]:
from IPython.core.display import HTML


class Visualizer:
    def __init__(
        self,
        model: ModelBase,
        dataset: Dataset,
        item_data: str | None=None,
        items: pd.DataFrame | None = None,
        items_df_path: str = "../datasets/items.csv",
    ):
        if item_data is None:
            item_data = ["title"]

        self.model = model
        self.dataset = dataset
        if items is None:
            self.items = pd.read_csv(items_df_path)
        else:
            self.items = items
        for d in item_data:
            assert (
                d in self.items.columns
            ), f'Unknown column "{d}". Can only visualize information about {", ".join(self.items.columns)}.'
        self.item_data = item_data

    def display_item_data(self, interactions):
        return pd.merge(interactions, self.items)[["user_id"] + self.item_data]

    @staticmethod
    def pretty_print(df):
        display(HTML(df.to_html()))

    def get_reco(self, user_ids: list[int], k: int = 10):
        reco = self.model.recommend(np.array(user_ids), self.dataset, k, True)
        return self.display_item_data(reco)

    def get_history(self, user_ids: list[int]):
        df = self.dataset.interactions.df
        history = df[df.user_id.isin(user_ids)]
        return self.display_item_data(history)

    def analyse_recos(self, user_ids: list[int]):
        for user_id in user_ids:
            print(f"User {user_id} watched these film:")
            self.pretty_print(self.get_history([user_id]))
            print(f"And got these films as recomendations")
            self.pretty_print(self.get_reco([user_id]))
            print("\n")

In [12]:
model = RandomModel(random_state=32)
dataset = Dataset.construct(interactions_df)
model.fit(dataset)
USER_IDS = [1091234, 787802, 948921]

In [13]:
viz = Visualizer(model, dataset, item_data=["title", "genres"], items=items)
viz.analyse_recos(USER_IDS)

User 1091234 watched these film:


,user_id,title,genres


And got these films as recomendations


,user_id,title,genres
0,1091234,Возвращение Будулая,мелодрамы
1,1091234,Новые приключения Аладдина (жестовым языком),"зарубежные, комедии"
2,1091234,Пропавшая грамота,"фэнтези, комедии"
3,1091234,Братья вне игры,"драмы, спорт"
4,1091234,Фрилансеры,"криминал, детективы, драмы, зарубежные, боевики"
5,1091234,Алые паруса: Новая история,"комедии, мелодрамы"
6,1091234,Женщина в беде 3,"детективы, мелодрамы"
7,1091234,Гордость и предубеждение,"драмы, мелодрамы"
8,1091234,Болванчики,"мультфильм, приключения, комедии"
9,1091234,Избави нас от лукавого,"ужасы, триллеры, детективы"




User 787802 watched these film:


...

,user_id,title,genres
0,787802,Возвращение Будулая,мелодрамы
1,787802,Новые приключения Аладдина (жестовым языком),"зарубежные, комедии"
2,787802,Пропавшая грамота,"фэнтези, комедии"
3,787802,Братья вне игры,"драмы, спорт"
4,787802,Фрилансеры,"криминал, детективы, драмы, зарубежные, боевики"
5,787802,Алые паруса: Новая история,"комедии, мелодрамы"
6,787802,Женщина в беде 3,"детективы, мелодрамы"
7,787802,Гордость и предубеждение,"драмы, мелодрамы"
8,787802,Болванчики,"мультфильм, приключения, комедии"
9,787802,Избави нас от лукавого,"ужасы, триллеры, детективы"




User 948921 watched these film:


,user_id,title,genres
0,948921,Спирит Непокорный,"семейное, мультфильм, приключения"
1,948921,Медвежонок Винни и его друзья,"мюзиклы, мультфильм, приключения, комедии"


And got these films as recomendations


,user_id,title,genres
0,948921,Возвращение Будулая,мелодрамы
1,948921,Новые приключения Аладдина (жестовым языком),"зарубежные, комедии"
2,948921,Пропавшая грамота,"фэнтези, комедии"
3,948921,Братья вне игры,"драмы, спорт"
4,948921,Фрилансеры,"криминал, детективы, драмы, зарубежные, боевики"
5,948921,Алые паруса: Новая история,"комедии, мелодрамы"
6,948921,Женщина в беде 3,"детективы, мелодрамы"
7,948921,Гордость и предубеждение,"драмы, мелодрамы"
8,948921,Болванчики,"мультфильм, приключения, комедии"
9,948921,Избави нас от лукавого,"ужасы, триллеры, детективы"
